In [1]:
import requests
import pandas as pd
import urllib3
import time
from datetime import datetime
import os

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
# Dictionary to store names with their respective IP address, login, and password
credentials = {
    'Santa Joana':     {'ip': '10.250.113.97' , 'login': 'WEB', 'password': 'web'},
    'Vila Mariana':    {'ip': '10.250.115.39' , 'login': 'WEB', 'password': 'web'},
    'Ibirapuera':      {'ip': '10.250.110.208', 'login': 'WEB', 'password': 'web'},
    'Belo Horizonte':  {'ip': '192.168.215.43', 'login': 'WEB', 'password': 'web'},
    'Brasilia':        {'ip': '192.168.1.20'  , 'login': 'WEB', 'password': 'web'},
    'Salvador':          {'ip': '10.2.0.70'     , 'login': 'WEB', 'password': 'web'},
}

## Login

In [21]:
def get_with_auth(url, login, password):
    # Define the parameters to send in the GET request
    params = {
        'username': login,
        'password': password
    }

    try:
        # Send GET request with parameters
        response = requests.get(url, params=params, verify=False ) #, cert=("C:/Users/FilipeFurlanBellotti/Documents/Vitrolife/API/API Prototype/cert/cert.pem", "C:/Users/FilipeFurlanBellotti/Documents/Vitrolife/API/API Prototype/cert/key.pem"))
        
        # Raise an exception if the request failed
        response.raise_for_status()
        
        # Return the response content
        return response  # Use response.json() for JSON data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
        return None

In [22]:
base_url = f"https://{credentials['Ibirapuera']['ip']}:4000/"
    
login_url = "LOGIN"

URL = base_url + login_url

content = get_with_auth(URL, 'Web', 'web')

token = content.json()['Token']

token

'0CB80024B6EE44A19F3CCEC2038286B6'

## Patients

In [13]:
def get_patients(base_url, token):
    # Define the URL for getting patients
    url = base_url + "/GET/patients"

    # Define the headers with the API token
    headers = {
        'API-token': token
    }

    try:
        # Send GET request to fetch patients
        response = requests.get(url, headers=headers, verify=False)
        
        # Raise an exception if the request failed
        response.raise_for_status()
        
        # Return the response content
        return response  # Assuming the response is in JSON format
    except requests.exceptions.RequestException as e:
        print(f"Error fetching patients: {e}")
        return None

url = base_url + "/GET/patients"


response = get_patients(base_url, token)

if response is not None:
    # Convert the response to a DataFrame
    patients = response.json()
    df_patients = pd.DataFrame(patients['Patients'])



In [14]:
df_patients.head()

,PatientIDx,PatientID,FirstName,LastName,DateOfBirth
0,NEXTGEN_43025.6457683333,6789,mariana,NaN,NaN
1,S4AY087741064_43026.4644870949,4444,Nathali N,Nobrega,NaN
2,S4AY087786300_43026.6132430440,18102017,4 day,test,NaN
3,S4AY087745928_43026.6339803935,12345,Lala,NaN,NaN
4,S4AY087743901_43032.6947574768,50.690,"NEVES, PATRICIA FERRARI",07/05/1976,NaN


In [ ]:
patient_idx = df_patients[df_patients['PatientID'].str.contains('882')]#['PatientIDx'].values[0]
patient_idx

,PatientIDx,PatientID,FirstName,LastName,DateOfBirth
510,S4AY087759226_43552.5201770139,49.882,"FURCIN, SANDRA P. G.",13/11/1978,1978.11.01
2100,S4AY0877_44449.4519063542,88278,"CUNHA, LIVIA M.F.",08/07/1978,1978.07.08
2286,S4AY087751193_44299.2948619329,85.882,"SILVEIRA, ANA CAROLINA A. G.",22/04/1987,1987.04.01
3035,PC1P7BHG_44872.5319592361,162.882,"PORTEN, THAYS Q.N.",17/11/1978,1978.11.17
3360,S4AY0877_44429.4873114815,507.882,"MALAQUIAS, MONIQUE N. S.",03/07/1986,1986.07.03
3592,PC1P7BHG_45150.5508110532,163.882,"BALDISSERA, FABIANA",22/10/1980,1980.10.22
4247,S4AY087725919_43631.5260931829,68.882,"ROCHA, CAROLINA DA SILVA",08/10/1979,1979.10.08
4763,PC1P7BHG_45275.5618219560,810.882,"CARVAJAL, LILIANA M. G.",30/07/1988,1988.07.30
4925,PC1P7BHG_45682.4168382523,863.882,"BARRETO, LILAH M.",31/05/1984,1984.05.31
4995,PC1P7BHG_46036.4589183333,882.110,"COTO, PALOMA A. S.",16/01/1985,1985.01.16


: 

In [17]:
patient_idx = 'PC1P7BHG_45996.4079372338'
df_patients[df_patients['PatientIDx'] == patient_idx]

,PatientIDx,PatientID,FirstName,LastName,DateOfBirth


# Treatments

In [18]:
def get_treatments(base_url, token, patientIDx):
    """
    Retrieve treatments for a given patientIDx from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        patientIDx (str): The unique patient identifier.

    Returns:
        dict or None: The JSON response containing treatments, or None if request fails.
    """
    url = f"{base_url}/GET/TREATMENT"
    params = {'patientIDx': patientIDx}
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, params=params, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching treatments: {e}")
        return None

In [19]:
treatment_number = get_treatments(base_url, token, patient_idx)
print(treatment_number)

treatment_number = treatment_number['TreatmentList'][0]
treatment_number

Error fetching treatments: Expecting value: line 1 column 1 (char 0)
None


TypeError: 'NoneType' object is not subscriptable

In [26]:
# treatment_number = '2025-2774'

# Embryo ID

In [27]:
def get_embryo_id(base_url, token, patientIDx, treatment_name):
    """
    Retrieve embryo ID data for a given patientIDx and treatment name from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        patientIDx (str): The unique patient identifier.
        treatment_name (str): The treatment name.

    Returns:
        dict or None: The JSON response containing embryo ID data, or None if request fails.
    """
    url = f"{base_url}/GET/embryoID"
    params = {
        'PatientIDx': patientIDx,
        'TreatmentName': treatment_name
    }
    headers = {'API-token': token} 

    try:
        response = requests.get(url, headers=headers, params=params, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching embryo ID: {e}")
        return None

In [28]:
embryo_list = get_embryo_id(base_url, token, patient_idx, treatment_number)
embryo_list

{'EmbryoIDList': ['D2025.01.26_S03999_I3166_P-1',
  'D2025.01.26_S03999_I3166_P-2',
  'D2025.01.26_S03999_I3166_P-3',
  'D2025.01.26_S03999_I3166_P-4',
  'D2025.01.26_S03999_I3166_P-5',
  'D2025.01.26_S03999_I3166_P-6']}

In [29]:
embryo_idx = embryo_list['EmbryoIDList'][0]
embryo_idx

'D2025.01.26_S03999_I3166_P-1'

# Get Image

In [29]:
def save_embryo_image(response, embryo_id, image_run, focal_plane):
    """
    Save the image response to disk with ordered naming.
    
    Args:
        response: requests.Response object containing image data
        embryo_id (str): The EmbryoID
        image_run (int): The image run number
        focal_plane (int): The focal plane number
    """
    if response is None or response.status_code != 200:
        print("No valid response received")
        return
    
    # Create output directory if it doesn't exist
    image_dir = os.path.join('embryo_images', embryo_id)
    os.makedirs(image_dir, exist_ok=True)
    
    try:
        # Create filename with run, focal plane and ordering information
        # Adjust focal plane to be positive for sorting (e.g., -5 becomes 00, +5 becomes 10)
        adjusted_plane = focal_plane + 5
        image_filename = f"{image_run:04d}_{adjusted_plane:02d}_{embryo_id}_run{image_run}_focal{focal_plane}.jpg"
        image_path = os.path.join(image_dir, image_filename)
        
        # Save the image
        with open(image_path, 'wb') as f:
            f.write(response.content)
        print(f"Image saved to: {image_path}")
        
    except Exception as e:
        print(f"Error saving image: {e}")


In [30]:
def get_image(base_url, token, embryo_id, image_run, focal_plane=7):
    """
    Retrieve a specific image for an embryo from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        embryo_id (str): The EmbryoID.
        image_run (int): The image run number.
        focal_plane (int): The focal plane number (default=7).

    Returns:
        requests.Response or None: The response containing the image, or None if request fails.
    """
    url = f"{base_url}/GET/image"
    params = {
        'EmbryoID': embryo_id,
        'ImageRun': image_run,
        'focalPlane': focal_plane
    }
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, params=params, verify=False)
        response.raise_for_status()
        return response
    except requests.exceptions.RequestException as e:
        print(f"Error fetching image: {e}")
        return None

def save_embryo_image(response, embryo_id, image_run, focal_plane):
    """
    Save the image response to disk.
    
    Args:
        response: requests.Response object containing image data
        embryo_id (str): The EmbryoID
        image_run (int): The image run number
        focal_plane (int): The focal plane number
    """
    if response is None or response.status_code != 200:
        print("No valid response received")
        return
    
    # Create output directory if it doesn't exist
    image_dir = os.path.join('embryo_images', embryo_id, f'run_{image_run}')
    os.makedirs(image_dir, exist_ok=True)
    
    try:
        # Create filename with run and focal plane information
        image_filename = f"{embryo_id}_order_{focal_plane+6:02d}_focal{focal_plane}.jpg"
        image_path = os.path.join(image_dir, image_filename)
        
        # Save the image
        with open(image_path, 'wb') as f:
            f.write(response.content)
        print(f"Image saved to: {image_path}")
        
    except Exception as e:
        print(f"Error saving image: {e}")


In [ ]:
embryo_idx = embryo_list['EmbryoIDList'][0]
# embryo_idx = 'D2018.06.15_S00343_I3027_P-1'
# embryo_idx = 'D2018.04.25_S00244_I3027_P-9'
run = 650

# Example usage:
image_runs = range(run, run+1)  # Adjust range based on your needs
focal_planes = range(-5, 6)  # Adjust range based on your needs

for run in image_runs:
    for plane in focal_planes:
        response = get_image(base_url, token, embryo_idx, run, plane)
        if response is not None:
            save_embryo_image(response, embryo_idx, run, plane)
        time.sleep(0.1)  # Rate limiting

# Get ALL Images

In [6]:
def get_all_images(base_url, token, embryo_id, image_overlay=True):
    """
    Retrieve all images for a given EmbryoID from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        embryo_id (str): The EmbryoID.
        image_overlay (bool): Whether to include image overlay. Defaults to True.

    Returns:
        dict or None: The JSON response containing images data, or None if request fails.
    """
    url = f"{base_url}/GET/allimages"
    params = {
        'EmbryoID': embryo_id,
        'ImageOverlay': 'true', #str(image_overlay)
        'focalPlane': 0
    }
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, params=params, verify=False)
        response.raise_for_status()
        return response
    except requests.exceptions.RequestException as e:
        print(f"Error fetching images: {e}")
        return None

In [26]:
embryo_idx = 'D2025.12.05_S04471_I3027_P-1'
response = get_all_images(base_url, token, embryo_idx, image_overlay=True)
response

Error fetching images: 500 Server Error: Error during images lookup for url: https://10.250.110.208:4000//GET/allimages?EmbryoID=D2025.12.05_S04471_I3027_P-1&ImageOverlay=true&focalPlane=0


In [38]:
import io
from zipfile import ZipFile
import os
from datetime import datetime

def process_and_save_images(response, embryo_id):
    """
    Process a response containing a ZIP file of JPEG images and save them to disk.
    
    Args:
        response: requests.Response object containing ZIP data
        embryo_id: String containing the embryo ID to use in filenames
    """
    if response is None or response.status_code != 200:
        print("No valid response received")
        return
    
    # Create output directories if they don't exist
    zip_dir = 'embryo_zips'
    # image_dir = 'embryo_images'
    os.makedirs(zip_dir, exist_ok=True)
    # os.makedirs(image_dir, exist_ok=True)
    
    try:
        # Save the ZIP file first
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        zip_filename = f"{embryo_id}_{timestamp}.zip"
        zip_path = os.path.join(zip_dir, zip_filename)
        
        with open(zip_path, 'wb') as f:
            f.write(response.content)
        print(f"ZIP file saved to: {zip_path}")
        
        # # Now extract the images from the ZIP
        # with ZipFile(zip_path) as zip_file:
        #     # Process each file in the ZIP
        #     for filename in zip_file.namelist():
        #         if filename.lower().endswith(('.jpg', '.jpeg')):
        #             # Read the image data
        #             image_data = zip_file.read(filename)
                    
        #             # Create output filename using embryo_id
        #             output_filename = f"{embryo_id}_{filename}"
        #             output_path = os.path.join(image_dir, output_filename)
                    
        #             # Save the image
        #             with open(output_path, 'wb') as f:
        #                 f.write(image_data)
        
        # print(f"Images extracted to: {image_dir}")
        
    except Exception as e:
        print(f"Error processing ZIP file: {e}")

# Process and save the images from the response
process_and_save_images(response, embryo_idx)

No valid response received
